
## Execute SQL Server Remote Procedure

Notebook has a few pre-requisites:

1. Cluster must have the following installed: `com.azure:azure-core:1.44.1` and `com.azure:azure-identity:1.10.0` under Cluster Configuration -> Libraries -> Maven (Search)
2. App Registration with Client Secret created
3. Service Principal User created and given access within Azure SQL DB:
    ```sql
    USE XXXX-1;

    CREATE USER [marcin-dev-pbi] 
        FROM EXTERNAL PROVIDER;

    ALTER ROLE db_owner --or any role that applies to execute SP
        ADD MEMBER [marcin-dev-pbi];
    GRANT EXECUTE  
        ON SCHEMA::dbo  
        TO [marcin-dev-pbi];

    -- my test procedure, feel free to use yours.
    CREATE PROCEDURE dbo.usp_HelloSP
    AS
    BEGIN
        SELECT 'Hello from service principal!' AS Message;
    END;

    EXEC dbo.usp_HelloSP;
    ```
4. Credential created using Unity Catalog to pull from with Azure Databricks Connector.

In [0]:
%scala
import java.sql.DriverManager
import com.azure.identity.ClientSecretCredentialBuilder
import com.azure.core.credential.{TokenCredential, TokenRequestContext}
import com.databricks.dbutils_v1.DBUtilsHolder.dbutils
import java.sql.{DriverManager, ResultSet, CallableStatement}

// dbutils.widgets.removeAll()

// 2) Create three text widgets with optional defaults
dbutils.widgets.text("spScope",     "", "UC Credential")
dbutils.widgets.text("clientId",     "", "Client ID")
dbutils.widgets.text("clientSecret", "",                         "Client Secret")
dbutils.widgets.text("tenantId",     "", "Tenant ID")
dbutils.widgets.text("serverName", "",                         "Client Secret")
dbutils.widgets.text("databaseName",     "XXXX-1", "Tenant ID")

// 3) Read them back into your vals
val spScope     = dbutils.widgets.get("spScope")
val clientId     = dbutils.widgets.get("clientId")
val clientSecret = dbutils.widgets.get("clientSecret")
val tenantId     = dbutils.widgets.get("tenantId")
val serverName = dbutils.widgets.get("serverName")
val databaseName     = dbutils.widgets.get("databaseName")

import java.sql.DriverManager
import com.azure.identity.ClientSecretCredentialBuilder
import com.azure.core.credential.{TokenCredential, TokenRequestContext}
import com.databricks.dbutils_v1.DBUtilsHolder.dbutils
import java.sql.{DriverManager, ResultSet, CallableStatement}
spScope: String = ""
clientId: String = 
clientSecret: String = 
tenantId: String = 
serverName: String = 
databaseName: String =

In [0]:
%scala
val accessToken: String = try {
  // getServiceCredentialsProvider returns Object, so cast it
  val raw = dbutils.credentials.getServiceCredentialsProvider("my-sp-scope")
  val provider = raw.asInstanceOf[TokenCredential]  
  // build a request context and block for the token
  val req = new TokenRequestContext().addScopes(scope)
  val token = provider.getToken(req).block().getToken
  println("✅  ServiceCredentialProvider succeeded")
  token
} catch {
  case _: Throwable =>
    // fallback: hard-coded SP via Azure Identity SDK
    val cred = new ClientSecretCredentialBuilder()
      .clientId(clientId)
      .clientSecret(clientSecret)
      .tenantId(tenantId)
      .build()
    val req = new TokenRequestContext().addScopes(scope)
    val token = cred.getToken(req).block().getToken
    println("✅  ClientSecretCredential succeeded")
    token
}

// 3) Use it however you like
println(s"Got AAD token: ${accessToken.take(16)}…")
()

✅ ClientSecretCredential succeeded
Got AAD token: 
accessToken: String =

In [0]:
%scala
import java.util.Properties
val props = new Properties()
props.put("accessToken", accessToken)
props.put("encrypt", "true")
props.put("hostNameInCertificate", "*.database.windows.net")
props.put("loginTimeout", "30")

val jdbcUrl = s"jdbc:sqlserver://$serverName:1433;database=$databaseName"

import java.util.Properties
props: java.util.Properties = {encrypt=true, loginTimeout=30, accessToken=..-, hostNameInCertificate=*.database.windows.net}
jdbcUrl: String = jdbc:sqlserver://XXXX.database.windows.net:1433;database=XXXX-1

In [0]:
%scala
val query = "{call dbo.usp_HelloSP()}";  
val statement = DriverManager.getConnection(jdbcUrl, props).prepareCall(query);  

val rs: ResultSet = statement.executeQuery()

// 4) iterate
while (rs.next()) {
  // replace "ColName" with your actual column
  val col1 = rs.getString("Message")
  println(s"Got row: $col1")
}

statement.execute();
statement.close();

Got row: Hello from service principal!
query: String = {call dbo.usp_HelloSP()}
statement: java.sql.CallableStatement = SQLServerCallableStatement:6
rs: java.sql.ResultSet = SQLServerResultSet:7